In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

In [3]:
reviews = pd.read_csv('./Amsterdam/reviews2.csv')
listings_desc = pd.read_csv('./Amsterdam/listings2.csv')
listings = pd.read_csv('./Amsterdam/listings.csv')
neighbour = pd.read_csv('./Amsterdam/neighbourhoods.csv')
calendar = pd.read_csv('./Amsterdam/calendar.csv')

#C:\Users\avadi\Desktop\DataScience_Udacity\Project 1\Amsterdam

In [4]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,816940,Old South Lovely Garden Apartment,4290554,Joyce,NaN,Zuid,52.35323,4.85421,Entire home/apt,165,6,63,2022-11-28,0.54,1,11,2,03632A9BE56A736890D1
1,528022,Somewhere Else,2594559,Els,NaN,Westerpark,52.39114,4.88846,Private room,100,2,248,2022-11-28,3.30,1,212,37,0363 9289 A94D 5C21 A579
2,538723,Beautiful ground floor apartment,356740,Dimphy,NaN,Watergraafsmeer,52.35400,4.93965,Entire home/apt,140,5,57,2020-01-01,0.45,1,0,0,0363 D990 F92F DECF 88D4
3,549310,Family Apartment with Garden,2699033,Barbara,NaN,De Baarsjes - Oud-West,52.37126,4.86241,Entire home/apt,200,2,54,2022-08-10,0.60,1,7,7,0363 35CD 76C9 04D4 1A21
4,553514,"B&B WestergasTverblijf , Cosy!",618589,Mirjam,NaN,Westerpark,52.38444,4.86916,Entire home/apt,80,2,162,2020-02-16,1.34,1,1,0,0363 AD82 ADCF 17F3 C3EA


## Pre-process Data 

### Calendar Data

In [5]:
# Looking for nulls
calendar.isnull().sum()

# Looking for listings where price is null
calendar[calendar.price.isnull()].listing_id.unique()


array([          12550549,           32234160,           37591658,
                 37614789, 720266014528319438,           18631508],
      dtype=int64)

In [40]:
# It is safe to drop all rows that have null values as there are very little days/listings where the price is missing
calendar_clean = calendar.dropna()

In [36]:
# Number listings in/around Amsterdam = 6809
len(calendar_clean.dropna().listing_id.unique().tolist())

6809

In [41]:
calendar_clean['available']  = np.where(calendar_clean['available']=='t' , 1, 0)
calendar_clean['year'] = pd.to_datetime(calendar_clean['date']).dt.year
calendar_clean['month'] = pd.to_datetime(calendar_clean['date']).dt.month
calendar_clean['day'] = pd.to_datetime(calendar_clean['date']).dt.day_name()

<ipython-input-41-58eccb52175d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar_clean['available']  = np.where(calendar_clean['available']=='t' , 1, 0)
<ipython-input-41-58eccb52175d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar_clean['year'] = pd.to_datetime(calendar_clean['date']).dt.year
<ipython-input-41-58eccb52175d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [42]:
x = ['listing_id', 'year', 'month', 'day', 'minimum_nights','maximum_nights']
calendar_clean[["price", "adjusted_price"]] = calendar_clean[["price", "adjusted_price"]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(pd.to_numeric)
calendar_clean = calendar_clean.groupby(x, as_index = False, group_keys = False).aggregate({'available':'sum','price':['mean'],'adjusted_price':'mean'})

<ipython-input-42-0dca80472e2f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  calendar_clean[["price", "adjusted_price"]] = calendar_clean[["price", "adjusted_price"]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(pd.to_numeric)
c:\Users\avadi\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [43]:
calendar_clean.head(20)

,listing_id,year,month,day,minimum_nights,maximum_nights,available,price,adjusted_price
,,,,,,,sum,mean,mean
0,2818,2022,12,Friday,3,1125,0,69.00,69.00
1,2818,2022,12,Monday,3,1125,1,59.00,59.00
2,2818,2022,12,Saturday,3,1125,0,69.25,69.25
3,2818,2022,12,Sunday,3,1125,0,59.00,59.00
4,2818,2022,12,Thursday,3,1125,0,69.00,69.00
5,2818,2022,12,Tuesday,3,1125,2,59.00,59.00
6,2818,2022,12,Wednesday,3,1125,2,69.00,69.00
7,2818,2023,1,Friday,3,1125,3,59.00,59.00
8,2818,2023,1,Monday,3,1125,3,67.00,67.00


In [50]:
#calendar.join(listings, index='listing_id', lsuffix="_left", rsuffix="_right")

#pd.merge(calendar_clean, listings, on=['listing_id','id'])

listings[listings['id'] == 2818]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
5,2818,Quiet Garden View Room & Super Fast Wi-Fi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.36435,4.94358,Private room,59,3,314,2022-11-06,1.88,1,27,29,0363 5F3A 5684 6750 D14D


In [33]:
calendar[calendar.listing_id == 2818]

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2818,2022-12-05,f,$59.00,$59.00,3,1125
1,2818,2022-12-06,t,$59.00,$59.00,3,1125
2,2818,2022-12-07,t,$59.00,$59.00,3,1125
3,2818,2022-12-08,f,$59.00,$59.00,3,1125
4,2818,2022-12-09,f,$59.00,$59.00,3,1125
...,...,...,...,...,...,...,...
360,2818,2023-11-30,f,$75.00,$75.00,3,1125
361,2818,2023-12-01,f,$76.00,$76.00,3,1125
362,2818,2023-12-02,f,$76.00,$76.00,3,1125
363,2818,2023-12-03,f,$76.00,$76.00,3,1125
